In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import nbformat as nbf
from pathlib import Path
from copy import deepcopy

# ✅ Update path for your new notebook
in_path = Path("/content/drive/MyDrive/Colab Notebooks/BERT_base_Supervised_Fine_Tuning.ipynb")
out_path = in_path.with_name(in_path.stem + "_clean.ipynb")

# Read notebook
with open(in_path, encoding="utf-8") as f:
    nb = nbf.read(f, as_version=4)

# 1) Drop broken widgets metadata
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

# 2) Strip widget-view outputs (cause GitHub render errors)
for cell in nb.cells:
    if cell.get("outputs"):
        new_outputs = []
        for o in cell["outputs"]:
            d = o.get("data", {})
            if isinstance(d, dict) and "application/vnd.jupyter.widget-view+json" in d:
                continue
            new_outputs.append(o)
        cell["outputs"] = new_outputs
    cell.get("metadata", {}).pop("execution", None)  # optional cleanup

# Save cleaned copy
nbf.write(nb, out_path)
print("✅ Cleaned notebook saved to:", out_path)


In [ ]:
pip install unsloth transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.8/184.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name =  "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length = 2048,
    load_in_4bit = True
)


==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model, r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

Unsloth 2025.8.9 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [ ]:
tokenizer = get_chat_template(tokenizer,chat_template= "llama-3.1")

In [ ]:
df = load_dataset("mlabonne/FineTome-100k")

In [ ]:
dataset = standardize_sharegpt(df['train'])

In [ ]:
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [ ]:
dataset = dataset.map(
    lambda examples: {
        "text":[
            tokenizer.apply_chat_template(convo, tokenize = False)
            for convo in examples["conversations"]
        ]
    },
    batched=True
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    # dataset_text_field = "text", # Commenting this out as formatting_func is required
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim= "paged_adamw_8bit"
    ),
    formatting_func=lambda example: example["text"],
)

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yashgamerz0007 (yashgamerz0007-sgu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.579900
2,2.002200
3,1.511800
4,1.523900
5,1.465400
6,1.574400
7,1.002400
8,1.660300
9,1.362100
10,1.393100


TrainOutput(global_step=60, training_loss=1.1570710917313893, metrics={'train_runtime': 297.1206, 'train_samples_per_second': 1.616, 'train_steps_per_second': 0.202, 'total_flos': 1902923896037376.0, 'train_loss': 1.1570710917313893})

In [ ]:
model.save_pretrained("finetune")

In [ ]:
interface_model, interface_tokenizer = FastLanguageModel.from_pretrained(
    model_name="./finetune",
    max_seq_length=2048,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
test_prompts = {
    "what are the key principles of investments"
}
for prompt in test_prompts:
  formatted_prompt = interface_tokenizer.apply_chat_template([{
      "role": "user",
      "content" : prompt
      }], tokenize=False)

  model_input = interface_tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

  generated_ids = interface_model.generate(
      **model_input,
      # max_temprature = 0.7, # Removed unsupported argument
      # do_samples=True, # Removed unsupported argument
      pad_token_id =interface_tokenizer.pad_token_id
  )
  response = interface_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 28 Aug 2025

user

what are the key principles of investmentsassistant

Key principles of investments include:

1. Diversification: Spreading investments across different asset classes, sectors, and geographic regions to minimize risk and maximize potential returns.
2. Asset allocation: Allocating investments to different asset classes based on risk tolerance, investment horizon, and financial goals.
3. Risk management: Identifying and mitigating potential risks, such as market volatility, liquidity, and credit risk.
4. Downtime management: Managing potential losses during market downturns or periods of low interest rates.
5. Regular portfolio rebalancing: Periodically reviewing and adjusting the investment portfolio to maintain its target asset allocation and risk level.
6. Tax efficiency: Minimizing taxes on investment returns by using tax-efficient investment strategies and strategies to reduce tax liabilities.
7. ESG (Enviro